In [1]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
import psycopg2
import datetime
import random
import os

In [2]:
load_dotenv()

True

In [3]:
df = pd.read_excel('Data Model - Pizza Sales.xlsx')

In [4]:
df['order_date'] = df['order_date'].astype(str)
df['order_time'] = df['order_time'].astype(str)

In [5]:
df['order_datetime'] = pd.to_datetime(df['order_date'] + ' ' + df['order_time'])
df['order_datetime']

0       2015-01-01 11:38:36
1       2015-01-01 11:57:40
2       2015-01-01 11:57:40
3       2015-01-01 11:57:40
4       2015-01-01 11:57:40
                ...        
48615   2015-12-31 21:23:10
48616   2015-12-31 21:23:10
48617   2015-12-31 21:23:10
48618   2015-12-31 22:09:54
48619   2015-12-31 23:02:05
Name: order_datetime, Length: 48620, dtype: datetime64[ns]

In [6]:
df['order_hour'] = df['order_datetime'].dt.hour

In [7]:
df['weekday'] = df['order_datetime'].dt.day_name()

# Group by 'weekday' and count the number of orders for each weekday
orders_per_weekday = df.groupby('weekday').size()

In [8]:
df['order_hour'] = df['order_datetime'].dt.hour

df['order_weekday'] = df['order_datetime'].dt.day_name()

orders_per_hour_weekday = df.groupby(['order_hour', 'order_weekday']).size().reset_index(name='order_count')

orders_per_hour_weekday

,order_hour,order_weekday,order_count
0,9,Tuesday,4
1,10,Monday,3
2,10,Saturday,3
3,10,Sunday,2
4,10,Thursday,5
...,...,...,...
90,23,Friday,26
91,23,Monday,4
92,23,Saturday,31
93,23,Sunday,4


In [9]:
orders_per_hour = df.groupby('order_hour').size()

In [10]:
orders_per_hour_weekday['original_order_count'] = orders_per_hour_weekday['order_count']

order_multiplier = round(random.uniform(1, 6), 6)
orders_per_hour_weekday['order_count'] = orders_per_hour_weekday['original_order_count'] * order_multiplier

In [11]:
orders_per_hour_weekday['order_count'] = orders_per_hour_weekday['original_order_count'].astype('int32')

In [12]:
orders_per_hour_weekday[
    (orders_per_hour_weekday['order_weekday'] == datetime.datetime.now().strftime("%A") )
    &
    (orders_per_hour_weekday['order_hour'] == datetime.datetime.now().hour)
]['order_count']

54    757
Name: order_count, dtype: int32

In [38]:
import numpy as np

# Your target number
target_number = orders_per_hour_weekday[
    # (orders_per_hour_weekday['order_weekday'] == datetime.datetime.now().strftime("%A"))
    (orders_per_hour_weekday['order_weekday'] == "Tuesday")
    &
    # (orders_per_hour_weekday['order_hour'] == datetime.datetime.now().hour)
    (orders_per_hour_weekday['order_hour'] == 10)
]['order_count'].max()

target_number = target_number if not np.isnan(target_number) else random.randint(0, 300)

# Standard deviation (smaller values mean the number is more likely to be close to the target)
std_dev = int(target_number * .3)

# Generate a random number close to the target number
random_number = np.random.normal(loc=target_number, scale=std_dev)

# If you need the number to be an integer, you can round it
random_number_rounded = int(round(random_number))

print(random_number)  # This will print the generated number
print(random_number_rounded)  # This will print the rounded number

193.7742890045244
194


In [15]:
# Number of seconds
total_seconds = 3600

# Generate a list of random timestamps
timestamps = [random.randint(0, total_seconds-1) for _ in range(random_number_rounded)]

# Sort the timestamps in ascending order
timestamps.sort()

# Print the timestamps
print(timestamps, len(timestamps))

[1, 5, 16, 35, 46, 47, 49, 51, 54, 55, 57, 59, 61, 62, 66, 67, 72, 73, 74, 77, 80, 100, 104, 106, 112, 113, 123, 137, 140, 164, 169, 170, 173, 180, 188, 189, 189, 191, 191, 192, 206, 210, 214, 218, 221, 225, 229, 230, 236, 245, 246, 251, 255, 257, 260, 265, 276, 278, 279, 299, 304, 305, 314, 322, 333, 334, 335, 337, 340, 343, 350, 354, 357, 382, 382, 385, 385, 406, 412, 413, 414, 416, 425, 430, 433, 445, 446, 448, 451, 461, 474, 496, 497, 503, 505, 507, 510, 516, 529, 530, 534, 538, 540, 547, 552, 557, 565, 570, 572, 575, 582, 586, 597, 600, 611, 613, 614, 617, 622, 622, 631, 636, 639, 642, 662, 665, 676, 676, 679, 698, 700, 704, 736, 737, 740, 762, 793, 794, 795, 814, 817, 827, 838, 847, 849, 859, 860, 866, 866, 867, 871, 876, 890, 895, 902, 905, 905, 909, 910, 924, 925, 939, 948, 953, 954, 970, 972, 973, 976, 979, 985, 998, 999, 1000, 1016, 1018, 1020, 1026, 1033, 1044, 1054, 1059, 1060, 1063, 1068, 1075, 1075, 1081, 1081, 1096, 1102, 1107, 1114, 1118, 1123, 1128, 1130, 1133, 1136, 1

In [16]:
from multiprocessing import Queue

q = Queue()
for t in timestamps:
    q.put(t)
    
q.qsize()

636

In [19]:
current_hour = datetime.datetime.now().strftime("%H")

if current_hour > 9 and current_hour <= 23:
    print('Open')

'17'

In [42]:
import datetime

current_hour = datetime.datetime.now().strftime("%H")
current_hour = datetime.datetime.strptime(current_hour, "%H")
new_hour = current_hour + datetime.timedelta(hours=7)
new_hour = new_hour.strftime("%H")
new_hour

'00'

In [172]:
3600 - (int(datetime.datetime.now().strftime("%M")) * 60 + int(datetime.datetime.now().strftime("%S")))

897